# 1 Library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer

# 2 Loading the Data & Preprocessing

In [28]:
# trim tail: to reduce the influence of extreme values
def trim_tail(data, lower_percentile, upper_percentile):
  low, high = np.percentile(data, [lower_percentile, upper_percentile])
  return np.clip(data, low, high)

def do_knn_impute(train_data, test_data, neighbors):
  # knn impute
  knn_imputer     = KNNImputer(n_neighbors=neighbors)
  train_data_filled  = knn_imputer.fit_transform(train_data)
  test_data_filled   = knn_imputer.transform(test_data)

  # transform to df
  columns_lst       = list(train_data.columns)
  train_data     = pd.DataFrame(train_data_filled, columns = columns_lst, index=train_data.index)
  test_data      = pd.DataFrame(test_data_filled, columns = columns_lst, index=test_data.index)
  return train_data, test_data

# load_preprocess:
#  1. load the data
#  2. Split the DE & FR
#  3. KNN imputer
#  4. trim tail on certain features
def load_preprocess(separate_country=False):
  # 1. Load train and set
  X_train     = pd.read_csv('X_train.csv', index_col='ID').drop(columns=["DAY_ID"])
  Y_train     = pd.read_csv('y_train.csv', index_col='ID')
  X_test      = pd.read_csv('X_test.csv', index_col='ID').drop(columns=["DAY_ID"])
  Y_test      = pd.read_csv('y_test_random_final.csv', index_col='ID')

  if separate_country:
      #   Join features and target for preprocessing
      train_df    = X_train.join(Y_train)
      test_df     = X_test.join(Y_test)

      # 2. Split training data into DE and FR datasets | trim tail
      train_fr    = train_df[train_df.COUNTRY=='FR'].sort_index().drop(columns=["COUNTRY"])
      train_de    = train_df[train_df.COUNTRY=='DE'].sort_index().drop(columns=["COUNTRY"])

      #   Split test data into DE and FR datasets | trim tail
      test_fr     = test_df[test_df.COUNTRY=='FR'].sort_index().drop(columns=["COUNTRY"])
      test_de     = test_df[test_df.COUNTRY=='DE'].sort_index().drop(columns=["COUNTRY"])

      # 3. KNN imputer
      print("KNN imputer started ...")
      train_fr, test_fr = do_knn_impute(train_fr, test_fr, 5)
      train_de, test_de = do_knn_impute(train_de, test_de, 5)
      print("KNN imputer end ! ")

      # 4. trim tail
      #   Choose the features that contain outliers
      trim_list = ['DE_CONSUMPTION', 'FR_CONSUMPTION',
      'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
      'DE_NET_IMPORT', 'FR_NET_IMPORT','DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD']
      print("Trim tail started, on columns: {}".format(trim_list))

      #   Trim the train data for FR & DE
      trimmed_train_fr = trim_tail(train_fr[trim_list], 1, 99)
      trimmed_train_de = trim_tail(train_de[trim_list], 1, 99)
      trimmed_test_fr  = trim_tail(test_fr[trim_list], 1, 99)
      trimmed_test_de  = trim_tail(test_de[trim_list], 1, 99)

      #   replace original features with trimed features
      train_fr[trim_list] = trimmed_train_fr
      train_de[trim_list] = trimmed_train_de
      test_fr[trim_list] = trimmed_test_fr
      test_de[trim_list] = trimmed_test_de

      #   seperate the features and target
      X_train_fr  = train_fr.drop(columns=['TARGET'])
      X_train_de  = train_de.drop(columns=['TARGET'])
      Y_train_fr  = train_fr[['TARGET']]
      Y_train_de  = train_de[['TARGET']]

      X_test_fr  = test_fr.drop(columns=['TARGET'])
      X_test_de  = test_de.drop(columns=['TARGET'])
      Y_test_fr  = test_fr[['TARGET']]
      Y_test_de  = test_de[['TARGET']]
      print("Proprocessing finished !")

      return [X_train_fr, Y_train_fr, X_train_de, Y_train_de, X_test_fr, Y_test_fr, X_test_de, Y_test_de]

  # If NOT separate country then return full train and test data
  else:
      ohc         = OneHotEncoder(drop='first')
      X_train['COUNTRY']  = ohc.fit_transform(X_train.COUNTRY.values.reshape(-1,1)).toarray()
      X_test['COUNTRY']   = ohc.fit_transform(X_test.COUNTRY.values.reshape(-1,1)).toarray()

      return X_train, Y_train, X_test, Y_test

## Seperate FR & DE

In [29]:
X_train_fr, Y_train_fr, X_train_de, Y_train_de, X_test_fr, Y_test_fr, X_test_de, Y_test_de = load_preprocess(separate_country=True)

KNN imputer started ...
KNN imputer end ! 
Trim tail started, on columns: ['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD']
Proprocessing finished !


## Not Seperate FR & DE

In [30]:
X_train, Y_train, X_test, Y_test = load_preprocess(separate_country=False)

# 3 Featur Engineering

## 3.1 Lag items:  built in-week lag items for DE & FR seperately

In [31]:
from pandas.errors import PerformanceWarning
import warnings

warnings.filterwarnings('ignore', category=PerformanceWarning)

def add_lagged_features(df):
    cols = ['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
    'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'GAS_RET', 'COAL_RET', 'CARBON_RET']

    lagged_features = pd.DataFrame(index=df.index)
    for col in cols:
        for i in range(1, 5):
            lagged_col_name = f'{col}_LAG{i}'
            lagged_features[lagged_col_name] = df[col].shift(i)

    df = pd.concat([df, lagged_features], axis=1)
    df.fillna(df.mean(), inplace=True)

    return df

## 3.2 Consumption Inspirations

In [32]:
def consumption_insp(df):
  # average commodity price variations
  df['avg_c_p_v']    = df[['GAS_RET', 'COAL_RET',	'CARBON_RET']].sum(axis=1).mean()
  df['avg_c_p_v_ma5']  = df['avg_c_p_v'].rolling(window=5).mean()
  df['avg_c_p_v_ma10']  = df['avg_c_p_v'].rolling(window=10).mean()

  # nuclear ratio trend
  epsilon = 1e-8
  df['nuclear_ratio_fr'] = df['FR_NUCLEAR'] / (df[['FR_GAS', 'FR_COAL', 'FR_HYDRO', 'FR_NUCLEAR', 'FR_SOLAR', 'FR_WINDPOW']].sum(axis=1) + epsilon)
  df['nuclear_ratio_de'] = df['DE_NUCLEAR'] / (df[['DE_GAS', 'DE_COAL', 'DE_HYDRO', 'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW']].sum(axis=1) + epsilon)

  # new energy transform efficency
  df['hydro_rain_fr'] = df['FR_HYDRO'] / (df['FR_RAIN'].rolling(window=5).mean() + epsilon)
  df['hydro_rain_de'] = df['DE_HYDRO'] / (df['DE_RAIN'].rolling(window=5).mean() + epsilon)

  df['wind_windpow_fr'] = df['FR_WINDPOW'] / (df['FR_WIND'].rolling(window=2).mean() + epsilon)
  df['wind_windpow_de'] = df['DE_WINDPOW'] / (df['DE_WIND'].rolling(window=2).mean() + epsilon)

  # residual_load premium cost
  df['load_premium_cost_fr']  = df['FR_RESIDUAL_LOAD'] * (df['avg_c_p_v'] + epsilon)
  df['load_premium_cost_de']  = df['DE_RESIDUAL_LOAD'] * (df['avg_c_p_v'] + epsilon)
  df['import_cost_fr']  = df['FR_NET_IMPORT'] * (df['avg_c_p_v'] + epsilon)
  df['import_cost_de']  = df['DE_NET_IMPORT'] * (df['avg_c_p_v'] + epsilon)

  # standerdize the new features
  features_to_scale = ['hydro_rain_fr', 'hydro_rain_de', 'wind_windpow_fr', 'wind_windpow_de',
             'load_premium_cost_fr', 'load_premium_cost_de', 'import_cost_fr', 'import_cost_de']
  # initialize scaler
  scaler = StandardScaler()
  df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
  df.fillna(df.mean(), inplace=True)

  return df

# Seperated DE & FR data for Next Modeling


## FR

In [ ]:
X_train_fr = add_lagged_features(X_train_fr)
X_test_fr = add_lagged_features(X_test_fr)
X_train_fr = consumption_insp(X_train_fr)
X_test_fr = consumption_insp(X_test_fr)

Y_train_fr
Y_test_fr

## DE

In [ ]:
X_train_de = add_lagged_features(X_train_de)
X_test_de = add_lagged_features(X_test_de)
X_train_de = consumption_insp(X_train_de)
X_test_de = consumption_insp(X_test_de)

Y_train_de
Y_test_de

## Not Seperate

In [ ]:
X_train = add_lagged_features(X_train)
X_test = add_lagged_features(X_test)
X_train = consumption_insp(X_train)
X_test = consumption_insp(X_test)

Y_train
Y_test

# Modeling ToolKit

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

def metric_kit(y_test, y_pred):
  # show the MSE
  mse = mean_squared_error(y_test, y_pred)
  print(f" MSE: {mse}")

  # show the MAE
  mse = mean_absolute_error(y_test, y_pred)
  print(f" MAE: {mse}")

  # show the spearmanr
  spear = spearmanr(y_test, y_pred).correlation
  print('Spearman correlation - default: {:.1f}%'.format(100 * spear))

def modeling_pipeline(X_train, Y_train, model_regressor, param_grid):
  # train valid set split
  X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=2024)

  # model define
  model = model_regressor

  # hyperparameter tuning
  grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
  grid_search.fit(X_train, y_train)

  # get the best parameters
  print(f"best_params: {grid_search.best_params_}")

  # use best parameter to predict
  best_model = grid_search.best_estimator_
  y_pred_best = best_model.predict(X_val)

  # show the metrics
  metric_kit(y_val, y_pred_best)

  return best_model

# 4 Decision Tree Modeling

In [36]:
model_regressor = DecisionTreeRegressor(random_state=2024)
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],  # 对于较新版本的scikit-learn
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = DecisionTreeRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

best_params: {'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
 MSE: 1.1148123404153867
 MAE: 0.5510857896134241
Spearman correlation - default: 21.7%
best_params: {'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
 MSE: 0.9808938504781785
 MAE: 0.6362261728323381
Spearman correlation - default: 43.5%


In [45]:
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],  # 对于较新版本的scikit-learn
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
model_regressor = DecisionTreeRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

best_params: {'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
 MSE: 1.4075318814209123
 MAE: 0.6865750418754671
Spearman correlation - default: -1.8%


# 5 Random Forest Modeling

In [37]:
model_regressor = RandomForestRegressor(random_state=2024)
param_grid = {
    'n_estimators': [100],
    'max_depth': [None, 15, 25],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4]
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = RandomForestRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
 MSE: 0.987878190720214
 MAE: 0.518452050289056
Spearman correlation - default: 7.0%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
 MSE: 0.5354197519041137
 MAE: 0.47802824672598826
Spearman correlation - default: 57.4%


In [46]:
param_grid = {
    'n_estimators': [100],
    'max_depth': [None, 15, 25],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4]
}
model_regressor = RandomForestRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
 MSE: 1.1493312839664984
 MAE: 0.6328470432773299
Spearman correlation - default: 10.5%


# 6 Bagging

In [47]:
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
import xgboost as xgb

In [49]:
model_regressor = BaggingRegressor(random_state=2024)
param_grid = {
    'n_estimators'      : [100, 200],
    'max_samples'       : [0.5, 0.7, 1.0],
    'max_features'      : [0.5, 0.7, 1.0],
    'bootstrap'         : [True],
    'bootstrap_features': [False]
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = BaggingRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


best_params: {'bootstrap': True, 'bootstrap_features': False, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 200}
 MSE: 0.9870948515012374
 MAE: 0.5095648525197348
Spearman correlation - default: 13.1%


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


best_params: {'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 100}
 MSE: 0.5560701342404257
 MAE: 0.49512243617098145
Spearman correlation - default: 54.4%


In [50]:
param_grid = {
    'n_estimators'      : [100, 200],
    'max_samples'       : [0.5, 0.7, 1.0],
    'max_features'      : [0.5, 0.7, 1.0],
    'bootstrap'         : [True],
    'bootstrap_features': [False]
}
model_regressor = BaggingRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


best_params: {'bootstrap': True, 'bootstrap_features': False, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 200}
 MSE: 1.1335105417195022
 MAE: 0.6245125328606613
Spearman correlation - default: 14.1%


# 7 AdaBoost

In [51]:
model_regressor = AdaBoostRegressor(random_state=2024)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = AdaBoostRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.1, 'n_estimators': 100}
 MSE: 0.9996764966311997
 MAE: 0.48710182886718906
Spearman correlation - default: 1.0%


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.1, 'n_estimators': 100}
 MSE: 0.6264037921372445
 MAE: 0.545109722388572
Spearman correlation - default: 55.0%


In [52]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
}
model_regressor = AdaBoostRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.01, 'n_estimators': 50}
 MSE: 1.104825813959231
 MAE: 0.6077906679172808
Spearman correlation - default: 11.0%


# 8 Gradient Boost

In [53]:
model_regressor = GradientBoostingRegressor(random_state=2024)
param_grid = {
    'n_estimators': [50, 100, 150, 300],
    'learning_rate': [0.01, 0.05, 0.1],
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = GradientBoostingRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.01, 'n_estimators': 50}
 MSE: 0.9702509063251323
 MAE: 0.4610495130440358
Spearman correlation - default: -2.3%


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.05, 'n_estimators': 50}
 MSE: 0.5815684914213218
 MAE: 0.5014346516325487
Spearman correlation - default: 55.3%


In [54]:
param_grid = {
    'n_estimators': [50, 100, 150, 300],
    'learning_rate': [0.01, 0.05, 0.1],
}
model_regressor = GradientBoostingRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'learning_rate': 0.01, 'n_estimators': 50}
 MSE: 1.102957901463703
 MAE: 0.5974752622768578
Spearman correlation - default: 17.2%


# 9 Extra Tree

In [55]:
model_regressor = ExtraTreesRegressor(random_state=2024)
param_grid = {
    'n_estimators': [50, 100, 150, 300, 500],
    'max_depth': [None, 10, 20],
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = ExtraTreesRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': 20, 'n_estimators': 100}
 MSE: 1.0299490327632566
 MAE: 0.5581992411538986
Spearman correlation - default: 6.4%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': None, 'n_estimators': 500}
 MSE: 0.5415273246472665
 MAE: 0.4749256019357339
Spearman correlation - default: 59.5%


In [56]:
param_grid = {
    'n_estimators': [50, 100, 150, 300, 500],
    'max_depth': [None, 10, 20],
}
model_regressor = ExtraTreesRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


best_params: {'max_depth': 10, 'n_estimators': 500}
 MSE: 1.093453106619794
 MAE: 0.6041149929140268
Spearman correlation - default: 26.8%


# 10 XGB

In [57]:
model_regressor = xgb.XGBRegressor(random_state=2024)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7],
}
model_fr = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

model_regressor = xgb.XGBRegressor(random_state=2024)
model_de = modeling_pipeline(X_train_de, Y_train_de, model_regressor, param_grid)

best_params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
 MSE: 0.974274317321525
 MAE: 0.4646354567317139
Spearman correlation - default: -5.8%
best_params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}
 MSE: 0.5374254449517155
 MAE: 0.474040847024263
Spearman correlation - default: 58.4%


In [58]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7],
}
model_regressor = xgb.XGBRegressor(random_state=2024)
model = modeling_pipeline(X_train, Y_train, model_regressor, param_grid)

best_params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
 MSE: 1.1023341942163798
 MAE: 0.5958034559405495
Spearman correlation - default: 10.1%


# Submission ToolKit

In [59]:
# Y_test_fr[:] = model.predict(X_test_fr)

# Y_test_fr.to_csv('benchmark_qrt_fr.csv', header = True, index = True)